In [1]:
import pandas as pd
import os
import datetime as dt
import numpy as np
import pyproj

In [2]:
def load_labels_df(filename):
    df = pd.read_csv(filename, sep='\t')
    df['start_time'] = df['Start Time'].apply(lambda x: dt.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
    df['end_time'] = df['End Time'].apply(lambda x: dt.datetime.strptime(x, '%Y/%m/%d %H:%M:%S'))
    df['labels'] = df['Transportation Mode']
    df = df.drop(['End Time', 'Start Time', 'Transportation Mode'], axis=1)
    return df

In [3]:
geod = pyproj.Geod(ellps='WGS84')

def to_datetime(string):
    return dt.datetime.strptime(string, '%Y-%m-%d %H:%M:%S')

def calculate_distance(long1, lat1, long2, lat2):
    if lat1 == lat2 and long1 == long2:
        return 0
    if False in np.isfinite([long1, long2, lat1, lat2]):
        return np.nan
    if lat1 < -90 or lat1 > 90 or lat2 < -90 or lat2 > 90:
        #raise ValueError('The range of latitudes seems to be invalid.')
        return np.nan
    if long1 < -180 or long1 > 180 or long2 < -180 or long2 > 180:
        return np.nan
        #raise ValueError('The range of longitudes seems to be invalid.')
    angle1,angle2,distance = geod.inv(long1, lat1, long2, lat2)
    return distance

def calculate_velocity(distance, timedelta):
    if timedelta.total_seconds() == 0: return np.nan
    return distance / timedelta.total_seconds()

def calculate_acceleration(velocity, velocity_next_position, timedelta):
    delta_v = velocity_next_position - velocity
    if timedelta.total_seconds() == 0: return np.nan
    return delta_v / timedelta.total_seconds()

In [4]:
headers_trajectory = ['lat', 'long', 'null', 'altitude','timestamp_float', 'date', 'time']

def load_trajectory_df(full_filename):
    subfolder = full_filename.split('/')[-3]
    trajectory_id = full_filename.split('/')[-1].split('.')[0]
    
    df = pd.read_csv(full_filename, skiprows = 6, header = None, names = headers_trajectory)
   
    df['datetime'] = df.apply(lambda z: dt.datetime.strptime(z.date + ' ' + z.time, '%Y-%m-%d %H:%M:%S'), axis=1)
    df['datetime_next_position'] = df['datetime'].shift(-1)
    df['timedelta'] = df.apply(lambda z: z.datetime_next_position - z.datetime, axis=1)
    df = df.drop(['datetime_next_position'], axis=1)
    df = df.drop(['null', 'timestamp_float', 'date', 'time'], axis=1)
    
    
    df['long_next_position'] = df['long'].shift(-1)
    df['lat_next_position'] = df['lat'].shift(-1)
    df['distance'] = df.apply(lambda z: calculate_distance(z.long, z.lat, z.long_next_position, z.lat_next_position), axis=1)
    df = df.drop(['long_next_position', 'lat_next_position'], axis=1)
    
    df['velocity'] = df.apply(lambda z: calculate_velocity(z.distance, z.timedelta), axis=1)
    df['velocity_next_position'] = df['velocity'].shift(-1)
    df['acceleration'] = df.apply(lambda z: calculate_acceleration(z.velocity, z.velocity_next_position, z.timedelta), axis=1)
    df = df.drop(['velocity_next_position'], axis=1)
    
    df['trajectory_id'] = trajectory_id
    df['subfolder'] = subfolder
    df['labels'] = ''
    #calculate_agg_features(df)
    return df

In [14]:
data_path = 'Data/'
labels_file = 'labels.txt'
output_folder = 'processed_data/'
trajectory_folder = 'Trajectory/'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)
directories = os.listdir(data_path)
count = 0

for subfolder in directories:
    df_traj_all = []
    list_df_traj = []
    traj_folder = data_path + subfolder + '/' + trajectory_folder
    traj_files = os.listdir(traj_folder)
    number = subfolder
    traj_files_full_path = [traj_folder + traj_file for traj_file in traj_files]
    subfolder = data_path + subfolder + '/'
    
    # Which files are actually labelled?
    if(labels_file in os.listdir(subfolder)):
        label_path = os.path.join(subfolder + labels_file)
        print(label_path)
        df_labels = load_labels_df(label_path)
        print(df_labels.head())

        for file in traj_files_full_path:
#             list_df_traj.append(load_trajectory_df(file))
            print(count, file)
            count = count + 1
        df_traj_all = pd.concat(list_df_traj)
        list_df_traj = []
        
        for trip in df_labels.index.values:
            start_time = df_labels.iloc[trip]['start_time']
            end_time = df_labels.iloc[trip]['end_time']
            labels = df_labels.iloc[trip]['labels']
            print(labels)
            # If label exists in label file
            if labels:
                print("label found")
                df_traj_all.loc[(df_traj_all['datetime'] >= start_time) & 
                                (df_traj_all['datetime'] <= end_time), 'labels'] = labels

    output_filename = output_folder + number + '.csv'
    print(output_filename)
    df_traj_all.to_csv(output_filename, sep=",")
    del df_traj_all
    


Data/010/labels.txt
           start_time            end_time labels
0 2007-06-26 11:32:29 2007-06-26 11:40:29    bus
1 2008-03-28 14:52:54 2008-03-28 15:59:59  train
2 2008-03-28 16:00:00 2008-03-28 22:02:00  train
3 2008-03-29 01:27:50 2008-03-29 15:59:59  train
4 2008-03-29 16:00:00 2008-03-30 15:59:59  train
0 Data/010/Trajectory/20070804033032.plt
1 Data/010/Trajectory/20070804155303.plt
2 Data/010/Trajectory/20070805070503.plt
3 Data/010/Trajectory/20070828171302.plt
4 Data/010/Trajectory/20070830203928.plt
5 Data/010/Trajectory/20070901022340.plt
6 Data/010/Trajectory/20070903095208.plt
7 Data/010/Trajectory/20070905163053.plt
8 Data/010/Trajectory/20070906204521.plt
9 Data/010/Trajectory/20070907075003.plt
10 Data/010/Trajectory/20070908081710.plt
11 Data/010/Trajectory/20070910074631.plt
12 Data/010/Trajectory/20070910204430.plt
13 Data/010/Trajectory/20070919121546.plt
14 Data/010/Trajectory/20070919122147.plt
15 Data/010/Trajectory/20070920074804.plt
16 Data/010/Trajectory/2

586 Data/062/Trajectory/20080701102103.plt
587 Data/062/Trajectory/20080701140204.plt
588 Data/062/Trajectory/20080703095601.plt
589 Data/062/Trajectory/20080703122722.plt
590 Data/062/Trajectory/20080706105319.plt
591 Data/062/Trajectory/20080707011746.plt
592 Data/062/Trajectory/20080707174201.plt
593 Data/062/Trajectory/20080708102528.plt
594 Data/062/Trajectory/20080709133136.plt
595 Data/062/Trajectory/20080710101047.plt
596 Data/062/Trajectory/20080710133937.plt
597 Data/062/Trajectory/20080711024944.plt
598 Data/062/Trajectory/20080711081701.plt
599 Data/062/Trajectory/20080713054527.plt
600 Data/062/Trajectory/20080713064801.plt
601 Data/062/Trajectory/20080714090806.plt
602 Data/062/Trajectory/20080714131246.plt
603 Data/062/Trajectory/20080714132423.plt
604 Data/062/Trajectory/20080714134828.plt
605 Data/062/Trajectory/20080715011016.plt
606 Data/062/Trajectory/20080715024302.plt
607 Data/062/Trajectory/20080716011241.plt
608 Data/062/Trajectory/20080716100537.plt
609 Data/06

           start_time            end_time labels
0 2008-05-19 16:25:54 2008-05-19 16:29:22   walk
1 2008-05-21 02:21:37 2008-05-21 02:25:04   walk
2 2008-05-22 01:30:09 2008-05-22 01:33:51   walk
3 2008-05-29 01:55:59 2008-05-29 01:59:25   walk
4 2008-05-30 12:11:16 2008-05-30 12:49:49    bus
1490 Data/073/Trajectory/20080519030703.plt
1491 Data/073/Trajectory/20080519162554.plt
1492 Data/073/Trajectory/20080521022137.plt
1493 Data/073/Trajectory/20080521155907.plt
1494 Data/073/Trajectory/20080522013009.plt
1495 Data/073/Trajectory/20080523095839.plt
1496 Data/073/Trajectory/20080524012610.plt
1497 Data/073/Trajectory/20080524012842.plt
1498 Data/073/Trajectory/20080528024310.plt
1499 Data/073/Trajectory/20080529015559.plt
1500 Data/073/Trajectory/20080529155904.plt
1501 Data/073/Trajectory/20080530121116.plt
1502 Data/073/Trajectory/20080531015238.plt
1503 Data/073/Trajectory/20080531094402.plt
1504 Data/073/Trajectory/20080603163433.plt
1505 Data/073/Trajectory/20080605162539.plt
15

1894 Data/084/Trajectory/20080915121124.plt
1895 Data/084/Trajectory/20080916093612.plt
1896 Data/084/Trajectory/20080917130606.plt
1897 Data/084/Trajectory/20080917225206.plt
1898 Data/084/Trajectory/20080918103406.plt
1899 Data/084/Trajectory/20080918235405.plt
1900 Data/084/Trajectory/20080919234530.plt
1901 Data/084/Trajectory/20080920074829.plt
1902 Data/084/Trajectory/20080921233542.plt
1903 Data/084/Trajectory/20080922233154.plt
1904 Data/084/Trajectory/20080923073930.plt
1905 Data/084/Trajectory/20080924002807.plt
1906 Data/084/Trajectory/20080924111634.plt
1907 Data/084/Trajectory/20080924234553.plt
1908 Data/084/Trajectory/20080924235101.plt
1909 Data/084/Trajectory/20080925082636.plt
1910 Data/084/Trajectory/20080926070824.plt
1911 Data/084/Trajectory/20080928002406.plt
1912 Data/084/Trajectory/20080930020723.plt
1913 Data/084/Trajectory/20081006233706.plt
1914 Data/084/Trajectory/20081007094130.plt
1915 Data/084/Trajectory/20081008045306.plt
1916 Data/084/Trajectory/2008100

           start_time            end_time labels
0 2008-03-19 07:11:47 2008-03-19 07:56:22    bus
1 2008-03-22 00:25:17 2008-03-22 00:33:32   walk
2 2008-03-22 04:33:35 2008-03-22 04:39:24   walk
3 2008-03-22 04:39:26 2008-03-22 05:17:19    bus
4 2008-03-24 00:27:42 2008-03-24 00:42:08   walk
2642 Data/092/Trajectory/20071130143600.plt
2643 Data/092/Trajectory/20071201092119.plt
2644 Data/092/Trajectory/20071202154208.plt
2645 Data/092/Trajectory/20071204160407.plt
2646 Data/092/Trajectory/20071205144716.plt
2647 Data/092/Trajectory/20071207092349.plt
2648 Data/092/Trajectory/20071208122242.plt
2649 Data/092/Trajectory/20071210083533.plt
2650 Data/092/Trajectory/20071211164301.plt
2651 Data/092/Trajectory/20071214113858.plt
2652 Data/092/Trajectory/20071215123448.plt
2653 Data/092/Trajectory/20071216083905.plt
2654 Data/092/Trajectory/20071216153038.plt
2655 Data/092/Trajectory/20071217103746.plt
2656 Data/092/Trajectory/20071218083254.plt
2657 Data/092/Trajectory/20071218181126.plt
26

...

           start_time            end_time labels
0 2007-07-24 22:00:00 2007-07-25 07:16:53   taxi
1 2007-07-25 07:16:55 2007-07-25 08:52:19   walk
2 2007-07-25 09:45:53 2007-07-25 10:30:00   taxi
3 2007-07-25 12:32:00 2007-07-25 14:13:38   walk
4 2007-07-25 22:59:00 2007-07-26 13:43:00   walk
7491 Data/153/Trajectory/20070721050708.plt
7492 Data/153/Trajectory/20070722034529.plt
7493 Data/153/Trajectory/20070724234145.plt
7494 Data/153/Trajectory/20070726053643.plt
7495 Data/153/Trajectory/20070727034219.plt
7496 Data/153/Trajectory/20070728035217.plt
7497 Data/153/Trajectory/20070728225925.plt
7498 Data/153/Trajectory/20080405011127.plt
7499 Data/153/Trajectory/20080417004937.plt
7500 Data/153/Trajectory/20080418004452.plt
7501 Data/153/Trajectory/20080418114105.plt
7502 Data/153/Trajectory/20080421005421.plt
7503 Data/153/Trajectory/20080426123357.plt
7504 Data/153/Trajectory/20080429084657.plt
7505 Data/153/Trajectory/20080430090427.plt
7506 Data/153/Trajectory/20080501023630.plt
75

8545 Data/153/Trajectory/20100609235222.plt
8546 Data/153/Trajectory/20100610102828.plt
8547 Data/153/Trajectory/20100610235304.plt
8548 Data/153/Trajectory/20100611235258.plt
8549 Data/153/Trajectory/20100612114303.plt
8550 Data/153/Trajectory/20100612235228.plt
8551 Data/153/Trajectory/20100614102634.plt
8552 Data/153/Trajectory/20100615021033.plt
8553 Data/153/Trajectory/20100616234428.plt
8554 Data/153/Trajectory/20100617094104.plt
8555 Data/153/Trajectory/20100617235133.plt
8556 Data/153/Trajectory/20100618144528.plt
8557 Data/153/Trajectory/20100620234933.plt
8558 Data/153/Trajectory/20100621115152.plt
8559 Data/153/Trajectory/20100622101428.plt
8560 Data/153/Trajectory/20100623120104.plt
8561 Data/153/Trajectory/20100623235234.plt
8562 Data/153/Trajectory/20100624111546.plt
8563 Data/153/Trajectory/20100624235428.plt
8564 Data/153/Trajectory/20100625111304.plt
8565 Data/153/Trajectory/20100626030834.plt
8566 Data/153/Trajectory/20100627235604.plt
8567 Data/153/Trajectory/2010062

           start_time            end_time labels
0 2007-04-12 10:21:16 2007-04-12 14:56:56   bike
1 2007-04-12 10:22:03 2007-04-12 10:26:25   walk
2 2007-04-13 00:53:06 2007-04-13 01:06:15   bike
3 2007-04-13 05:19:33 2007-04-13 05:27:34   taxi
4 2007-04-13 07:01:26 2007-04-13 07:18:02   taxi
9546 Data/161/Trajectory/20070412101853.plt
9547 Data/161/Trajectory/20070412102325.plt
9548 Data/161/Trajectory/20070413105648.plt
9549 Data/161/Trajectory/20070413150314.plt
9550 Data/161/Trajectory/20070414022829.plt
9551 Data/161/Trajectory/20070416130252.plt
9552 Data/161/Trajectory/20070417013015.plt
9553 Data/161/Trajectory/20070417130153.plt
9554 Data/161/Trajectory/20070419131511.plt
9555 Data/161/Trajectory/20070420023557.plt
9556 Data/161/Trajectory/20070424041508.plt
9557 Data/161/Trajectory/20070424124908.plt
9558 Data/161/Trajectory/20070425021220.plt
9559 Data/161/Trajectory/20070427020730.plt
9560 Data/161/Trajectory/20070427135340.plt
9561 Data/161/Trajectory/20070428134038.plt
Da

10326 Data/163/Trajectory/20120629074823.plt
10327 Data/163/Trajectory/20120629212834.plt
10328 Data/163/Trajectory/20120701174206.plt
10329 Data/163/Trajectory/20120702095658.plt
10330 Data/163/Trajectory/20120702215854.plt
10331 Data/163/Trajectory/20120703090300.plt
10332 Data/163/Trajectory/20120703211634.plt
10333 Data/163/Trajectory/20120704092036.plt
10334 Data/163/Trajectory/20120704203006.plt
10335 Data/163/Trajectory/20120705094324.plt
10336 Data/163/Trajectory/20120705210536.plt
10337 Data/163/Trajectory/20120706200734.plt
10338 Data/163/Trajectory/20120708094737.plt
10339 Data/163/Trajectory/20120709093049.plt
10340 Data/163/Trajectory/20120709204900.plt
10341 Data/163/Trajectory/20120710073254.plt
10342 Data/163/Trajectory/20120710210424.plt
10343 Data/163/Trajectory/20120711090736.plt
10344 Data/163/Trajectory/20120712091004.plt
10345 Data/163/Trajectory/20120712201303.plt
10346 Data/163/Trajectory/20120713090334.plt
10347 Data/163/Trajectory/20120713205006.plt
10348 Data

           start_time            end_time  labels
0 2008-08-20 12:09:17 2008-08-20 12:45:05    walk
1 2008-08-20 12:45:05 2008-08-20 13:00:25  subway
2 2008-08-20 13:00:25 2008-08-20 13:07:25    walk
3 2008-08-20 13:07:25 2008-08-20 13:12:59     bus
4 2008-08-20 13:13:59 2008-08-20 13:24:23    walk
10835 Data/179/Trajectory/20080820120904.plt
10836 Data/179/Trajectory/20080821082136.plt
10837 Data/179/Trajectory/20080821104940.plt
10838 Data/179/Trajectory/20080822033436.plt
10839 Data/179/Trajectory/20080823012829.plt
10840 Data/179/Trajectory/20080823121859.plt
10841 Data/179/Trajectory/20080824015336.plt
10842 Data/179/Trajectory/20080824114959.plt
10843 Data/179/Trajectory/20080827093230.plt
10844 Data/179/Trajectory/20080828021406.plt
10845 Data/179/Trajectory/20080828125117.plt
10846 Data/179/Trajectory/20080830014705.plt
10847 Data/179/Trajectory/20080830120236.plt
10848 Data/179/Trajectory/20080831090654.plt
10849 Data/179/Trajectory/20080902102330.plt
10850 Data/179/Trajectory